In [2]:
import os
from docx import Document

# Read Document

In [3]:
doc_id = "1"
user_document = f"memorandum_{doc_id}.docx"
user_document_path = os.path.join("../sample_data", user_document)

document = Document(user_document_path)

In [11]:
document_text = ""
for p in document.paragraphs:
    document_text += p.text + "\n"

print(document_text)

							DRAFT - FOR DISCUSSION PURPOSES


Further to our arrangements, please find below our comments concerning personal income tax (hereinafter: PIT) as well as social security consequences of performing remote work in the territory of Poland based on foreign (_____) employment contract. 
Our comments are based on our understanding of the information provided to us in the e-mail correspondence with    LLP and relevant documents (Offer of employment dated 27th February 2020 and Position description: Director, Corporate Talent Acquisition).
Our analysis is based on:
The Act of 26 July 1991 on the Personal Income Tax (further: PIT Act);
Polish Social Security System Act of 13 October 1998;
Convention between The Republic of Poland and _____ for the avoidance of double taxation and the prevention of fiscal evasion with respect to taxes on income of 14 May 2012 (further: DTT);
The Social Security Agreement of 2 April 2008 between The Republic of Poland and _____;
and most recent approach o

# LLM

In [8]:
from openai import AzureOpenAI

from dotenv import load_dotenv
load_dotenv(override=True)

True

In [9]:
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-08-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)

In [21]:
summary_system_prompt = """
You are going to be provided with a text from a document.
Your output will be used for creating a pptx presentation, for that determine a title and subtitle of the presentation and information about each section of the document which will be the content for each slide of the presentation.

Consider the following guidlines:
- Title should be at most 5 words long
- Subtitle should be at most 10 words long
- Each summary include all important details of the section, each summary should be at most 2 paragraphs long.

Provide your respons having ONLY json object following the format:

{
    "title": <<Title of the presentation>>,
    "subtitle": <<Subtitle>>,
    "sections": 
    [
        {
            "section_title": <<Section Title>>,
            "section_content": <<Summarized section Content>>
        },
        {
            "section_title": <<Section Title>>,
            "section_content": <<Summarized section Content>>
        },
        ...
    ]
}
"""

In [22]:
summary_response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "system",
            "content": summary_system_prompt,
        },
        {
            "role": "user",
            "content": f"This is the document: {document_text}",
        },
    ],
)

In [23]:
import json

In [24]:
json_str = summary_response.choices[0].message.content
json_str = json_str.replace("```json", "").replace("```", "")
summary_dict = json.loads(json_str)

# PPTX Creation

In [25]:
from pptx import Presentation

## Slide 1

In [26]:
template_path = '../pptx_templates/Memo.pptx'
prs = Presentation(template_path)

# Slide title

In [27]:
slide = prs.slides[0]
slide.placeholders[0].text = summary_dict["title"]
slide.placeholders[1].text = summary_dict["subtitle"]

## Slide 2, 3 and 4

In [28]:
for i, section in enumerate(summary_dict["sections"][:3]):
    slide = prs.slides[i + 1]
    slide.placeholders[0].text = section["section_title"]
    slide.placeholders[15].text = section["section_content"]

In [29]:
prs.save('../output/test.pptx')